In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')


ParserError: ignored

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)['crew'].values

In [ ]:
credits.shape

In [ ]:
movies = pd.merge(movies , credits, on = 'title')

In [ ]:
movies.shape

In [ ]:
movies.head()

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)


In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

In [ ]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
def convert4(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 4:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert4)

movies.head()

In [ ]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
      break
  return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres'] =movies['genres'].apply(lambda x:[i.replace(" " , "")for i in x])

In [ ]:
movies['keywords'] =movies['keywords'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['cast'] =movies['cast'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['crew'] =movies['crew'].apply(lambda x:[i.replace(" " , "")for i in x])

In [ ]:
movies.head()


In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_dataframe = movies[['movie_id' , 'title' , 'tags']]

In [ ]:
new_dataframe

In [ ]:
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_dataframe['tags'][0]

In [ ]:
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:x.lower())

In [ ]:
new_dataframe.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 , stop_words='english')

vector = cv.fit_transform(new_dataframe['tags']).toarray()


In [ ]:
vector = cv.fit_transform(new_dataframe['tags']).toarray()

In [ ]:
new_dataframe['tags'][0]

In [ ]:
vector.shape

In [ ]:
vector[0]

In [ ]:
cv.get_feature_names()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y=[]

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver stephenlang jamescameron')

In [ ]:
new_dataframe['tags'] = new_dataframe['tags'].apply(stem)

In [ ]:
new_dataframe.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vector)

In [ ]:
cosine_similarity(vector).shape

In [ ]:
similarities = cosine_similarity(vector)

In [ ]:
similarities[0]

In [ ]:
def recommend(movie):
  movieindex = new_dataframe[new_dataframe['title'] == movie].index[0] 
  distances = similarities[movieindex]
  movie_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:7]

  for i in movie_list:
    print(new_dataframe.iloc[i[0]].title)



In [ ]:
recommend('The Amazing Spider-Man')